## Анализ температурных данных и мониторинг текущей температуры через OpenWeatherMap API

**Описание задания:**  
Вы аналитик в компании, занимающейся изучением климатических изменений и мониторингом температур в разных городах. Вам нужно провести анализ исторических данных о температуре для выявления сезонных закономерностей и аномалий. Также необходимо подключить API OpenWeatherMap для получения текущей температуры в выбранных городах и сравнить её с историческими данными.


### Цели задания:
1. Провести **анализ временных рядов**, включая:
   - Вычисление скользящего среднего и стандартного отклонения для сглаживания температурных колебаний.
   - Определение аномалий на основе отклонений температуры от $ \text{скользящее среднее} \pm 2\sigma $.
   - Построение долгосрочных трендов изменения температуры.
   - Любые дополнительные исследования будут вам в плюс.

2. Осуществить **мониторинг текущей температуры**:
   - Получить текущую температуру через OpenWeatherMap API.
   - Сравнить её с историческим нормальным диапазоном для текущего сезона.

3. Разработать **интерактивное приложение**:
   - Дать пользователю возможность выбрать город.
   - Отобразить результаты анализа температур, включая временные ряды, сезонные профили и аномалии.
   - Провести анализ текущей температуры в контексте исторических данных.


### Описание данных
Исторические данные о температуре содержатся в файле `temperature_data.csv`, включают:
  - `city`: Название города.
  - `timestamp`: Дата (с шагом в 1 день).
  - `temperature`: Среднесуточная температура (в °C).
  - `season`: Сезон года (зима, весна, лето, осень).

Код для генерации файла вы найдете ниже.

### Этапы выполнения

1. **Анализ исторических данных**:
   - Вычислить **скользящее среднее** температуры с окном в 30 дней для сглаживания краткосрочных колебаний.
   - Рассчитать среднюю температуру и стандартное отклонение для каждого сезона в каждом городе.
   - Выявить аномалии, где температура выходит за пределы $ \text{среднее} \pm 2\sigma $.
   - Попробуйте распараллелить проведение этого анализа. Сравните скорость выполнения анализа с распараллеливанием и без него.

2. **Мониторинг текущей температуры**:
   - Подключить OpenWeatherMap API для получения текущей температуры города. Для получения API Key (бесплатно) надо зарегистрироваться на сайте. Обратите внимание, что API Key может активироваться только через 2-3 часа, это нормально. Посему получите ключ заранее.
   - Получить текущую температуру для выбранного города через OpenWeatherMap API.
   - Определить, является ли текущая температура нормальной, исходя из исторических данных для текущего сезона.
   - Данные на самом деле не совсем реальные (сюрпрайз). Поэтому на момент эксперимента погода в Берлине, Каире и Дубае была в рамках нормы, а в Пекине и Москве аномальная. Протестируйте свое решение для разных городов.
   - Попробуйте для получения текущей температуры использовать синхронные и асинхронные методы. Что здесь лучше использовать?

3. **Создание приложения на Streamlit**:
   - Добавить интерфейс для загрузки файла с историческими данными.
   - Добавить интерфейс для выбора города (из выпадающего списка).
   - Добавить форму для ввода API-ключа OpenWeatherMap. Когда он не введен, данные для текущей погоды не показываются. Если ключ некорректный, выведите на экран ошибку (должно приходить `{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}`).
   - Отобразить:
     - Описательную статистику по историческим данным для города, можно добавить визуализации.
     - Временной ряд температур с выделением аномалий (например, точками другого цвета).
     - Сезонные профили с указанием среднего и стандартного отклонения.
   - Вывести текущую температуру через API и указать, нормальна ли она для сезона.

## Генерация данных

In [1]:
import pandas as pd
import numpy as np

from scripts import current_temp, analysis, async_current_temp
from utils import test_sync_temp, test_async_temp, test_sync_analysis, test_parallel_analysis

In [2]:
# Реальные средние температуры (примерные данные) для городов по сезонам
seasonal_temperatures = {
    "New York": {"winter": 0, "spring": 10, "summer": 25, "autumn": 15},
    "London": {"winter": 5, "spring": 11, "summer": 18, "autumn": 12},
    "Paris": {"winter": 4, "spring": 12, "summer": 20, "autumn": 13},
    "Tokyo": {"winter": 6, "spring": 15, "summer": 27, "autumn": 18},
    "Moscow": {"winter": -10, "spring": 5, "summer": 18, "autumn": 8},
    "Sydney": {"winter": 12, "spring": 18, "summer": 25, "autumn": 20},
    "Berlin": {"winter": 0, "spring": 10, "summer": 20, "autumn": 11},
    "Beijing": {"winter": -2, "spring": 13, "summer": 27, "autumn": 16},
    "Rio de Janeiro": {"winter": 20, "spring": 25, "summer": 30, "autumn": 25},
    "Dubai": {"winter": 20, "spring": 30, "summer": 40, "autumn": 30},
    "Los Angeles": {"winter": 15, "spring": 18, "summer": 25, "autumn": 20},
    "Singapore": {"winter": 27, "spring": 28, "summer": 28, "autumn": 27},
    "Mumbai": {"winter": 25, "spring": 30, "summer": 35, "autumn": 30},
    "Cairo": {"winter": 15, "spring": 25, "summer": 35, "autumn": 25},
    "Mexico City": {"winter": 12, "spring": 18, "summer": 20, "autumn": 15},
}

# Сопоставление месяцев с сезонами
month_to_season = {12: "winter", 1: "winter", 2: "winter",
                   3: "spring", 4: "spring", 5: "spring",
                   6: "summer", 7: "summer", 8: "summer",
                   9: "autumn", 10: "autumn", 11: "autumn"}

# Генерация данных о температуре
def generate_realistic_temperature_data(cities, num_years=10):
    dates = pd.date_range(start="2010-01-01", periods=365 * num_years, freq="D")
    data = []

    for city in cities:
        for date in dates:
            season = month_to_season[date.month]
            mean_temp = seasonal_temperatures[city][season]
            # Добавляем случайное отклонение
            temperature = np.random.normal(loc=mean_temp, scale=5)
            data.append({"city": city, "timestamp": date, "temperature": temperature})

    df = pd.DataFrame(data)
    df['season'] = df['timestamp'].dt.month.map(lambda x: month_to_season[x])
    return df

# Генерация данных
data = generate_realistic_temperature_data(list(seasonal_temperatures.keys()))
data.to_csv('temperature_data.csv', index=False)


## Анализ исторических данных

### Температуры для каждого сезона в каждом городе

Рассчитаем среднюю температуру и стандартное отклонение для каждого сезона в каждом городе

In [3]:
data.groupby(['city', 'season']).agg({'temperature': ('mean', 'std')})

temperature          
                             mean       std
city           season                      
Beijing        autumn   15.821265  4.971861
               spring   13.133265  5.095347
               summer   26.927165  5.100578
               winter   -1.726743  5.125945
Berlin         autumn   11.413522  4.964867
               spring    9.781418  4.848280
               summer   20.129910  4.942110
               winter   -0.247984  4.901361
Cairo          autumn   24.706609  5.109818
               spring   24.845424  5.120604
               summer   35.201019  4.968636
               winter   14.839215  4.957106
Dubai          autumn   30.017708  4.878068
               spring   29.637859  4.907604
               summer   39.964027  4.798981
               winter   19.908876  5.018619
London         autumn   11.771700  5.094544
               spring   10.949947  4.815036
               summer   18.175242  4.880203
               winter    4.799531  5.039399
Los Angeles    autumn   20.136928  5.092435
               spring   17.831045  5.064521
               summer   25.030278  4.857314
               winter   15.019733  4.791543
Mexico City    autumn   15.323339  4.994481
               spring   17.758796  5.019460
               summer   20.248207  5.293106
               winter   11.849130  5.265686
Moscow         autumn    7.845211  5.112778
               spring    5.242447  4.982885
               summer   17.846421  5.133337
               winter   -9.955241  5.068607
Mumbai         autumn   29.840180  4.887886
               spring   30.079961  5.102112
               summer   34.924435  5.014582
               winter   25.014734  5.014904
New York       autumn   15.195659  4.809125
               spring   10.067800  4.779512
               summer   24.997162  4.997121
               winter    0.240244  4.855918
Paris          autumn   13.162173  5.154244
               spring   12.198375  4.959135
               summer   19.955678  4.902371
               winter    4.196652  5.143237
Rio de Janeiro autumn   24.747007  4.975474
               spring   25.120224  5.054899
               summer   30.307513  5.008764
               winter   19.790959  5.220738
Singapore      autumn   27.015834  5.209366
               spring   27.925003  4.844612
               summer   28.270203  4.865309
               winter   27.365064  4.957615
Sydney         autumn   20.161858  4.986423
               spring   17.863994  4.976218
               summer   25.155679  5.143159
               winter   11.937121  5.197082
Tokyo          autumn   17.967897  4.913586
               spring   14.957259  5.009331
               summer   26.807097  5.089839
               winter    5.817091  4.740167

С помощью groupby узнали необходимую информацию; для интереса и ~~попытки заработать дополнительных баллов~~ дополнительного анализа здесь и далее будем пытается смотреть на еще некоторые показатели / пилить что-то еще

Например, в преддверии новогодних праздников выведем топ-3 города с самой теплой зимой (в среднем)


In [4]:
data[data['season'] == 'winter'].groupby(['city', 'season']).agg(
    {'temperature': 'mean'}).sort_values(by = 'temperature', ascending=False).head(3)

,,temperature
city,season,
Singapore,winter,27.365064
Mumbai,winter,25.014734
Dubai,winter,19.908876


Также глянем на самый стабильный город по температуре (наименьшее стандартное отклонение между сезонами)

In [5]:
grouped_data = data.groupby(['city', 'season']).agg({'temperature': 'mean'}).reset_index()

grouped_data.groupby('city').agg(
    {'temperature': 'std'}).sort_values(by = 'temperature').head(1)

,temperature
city,
Singapore,0.5608


В Сингапуре практически не отличается погода в течение сезонов - логично! Он еще и во втором рейтинге подряд топ-1

### Вычисление скользящего среднего

Узнаем скользящее среднее по городам

In [6]:
data['rolling_mean'] = data.groupby('city')['temperature'].rolling(window=30).mean().reset_index(level=0, drop=True)
data['rolling_std'] = data.groupby('city')['temperature'].rolling(window=7).std().reset_index(level=0, drop=True)

data[40:50] # для первых 30 временных отметок недостаточно данных для скользящего среднего
            # решил далее в функциях порог не снижать и просто их не использовать

,city,timestamp,temperature,season,rolling_mean,rolling_std
40,New York,2010-02-10,-2.320241,winter,0.302745,5.302739
41,New York,2010-02-11,2.540607,winter,0.430756,5.372809
42,New York,2010-02-12,5.911489,winter,0.539180,4.455389
43,New York,2010-02-13,-6.234762,winter,0.228495,5.073094
44,New York,2010-02-14,2.456590,winter,0.086863,4.741574
45,New York,2010-02-15,-5.453728,winter,-0.195654,4.961246
46,New York,2010-02-16,-1.750750,winter,-0.299367,4.493288
47,New York,2010-02-17,1.750607,winter,-0.343278,4.510977
48,New York,2010-02-18,0.226614,winter,-0.284645,4.366722
49,New York,2010-02-19,-5.469181,winter,-0.632763,3.669983


### Поиск аномалий

In [7]:
data['is_anomaly'] = (data['temperature'] > data['rolling_mean'] + 2 *
                      data['rolling_std']) | (data['temperature'] < data['rolling_mean'] - 2 * data['rolling_std'])

data[data['is_anomaly'] == True]

,city,timestamp,temperature,season,rolling_mean,rolling_std,is_anomaly
59,New York,2010-03-01,14.041789,spring,-0.231454,5.951106,True
63,New York,2010-03-05,12.885706,spring,1.093626,5.831846,True
67,New York,2010-03-09,13.721536,spring,2.619479,4.548147,True
68,New York,2010-03-10,14.139883,spring,2.866047,4.990587,True
75,New York,2010-03-17,22.992054,spring,4.928291,6.433864,True
...,...,...,...,...,...,...,...
54650,Mexico City,2019-09-21,7.263223,autumn,17.415403,5.060853,True
54664,Mexico City,2019-10-05,5.173077,autumn,14.988051,3.660783,True
54676,Mexico City,2019-10-17,6.462397,autumn,14.530646,3.655637,True
54707,Mexico City,2019-11-17,0.294538,autumn,16.014036,7.158249,True


Посмотрим, в каком городе было больше неожиданных с точки зрения погоды дней

In [8]:
print(
    f"Город с наибольшим количеством аномальных дней: {data.groupby('city')['is_anomaly'].sum().idxmax()} "
    f"({data.groupby('city')['is_anomaly'].sum().max()} дня)\n"
    f"Город с наименьшим количеством аномальных дней: {data.groupby('city')['is_anomaly'].sum().idxmin()} "
    f"({data.groupby('city')['is_anomaly'].sum().min()} дня)"
)

Город с наибольшим количеством аномальных дней: Beijing (429 дня)
Город с наименьшим количеством аномальных дней: Singapore (124 дня)


Сингапур снова топ-1!

### Исследование распараллеливания анализа

Обернем весь код выше в функцию (скрипт), дополним его кодом про определение тренда. Функция буудет лежать (как и все остальные скрипты) в scripts.py

In [9]:
analysis('Moscow') 

Текущий сезон для города Moscow: winter

Минимальная температура в сезоне: -27.876830626043148
Максимальная температура в сезоне: 5.757819626838792
Средняя температура в сезоне:  -10.308275957852874

Профиль текущего сезона города Moscow
         city  timestamp  temperature  season  rolling_mean  rolling_std  \
18245  Moscow 2019-12-25    -9.765358  winter     -9.291112     5.500929   
18246  Moscow 2019-12-26    -7.142475  winter     -9.284995     4.738856   
18247  Moscow 2019-12-27   -13.183317  winter     -9.317816     5.036299   
18248  Moscow 2019-12-28   -12.224927  winter     -9.583863     5.182668   
18249  Moscow 2019-12-29    -9.031708  winter     -9.708461     3.438407   

       is_anomaly                trend  
18245       False  Положительный тренд  
18246       False  Положительный тренд  
18247       False  Положительный тренд  
18248       False  Положительный тренд  
18249       False  Положительный тренд  


Немного корявый вывод: я изначально делал вывод с помощью display, а не print, но мультипроцессинг несовместим с юпитеровским display. Поэтому немного коряво вышло с print

Попробуем теперь **асинхронный метод** и сравним работоспобность, вызвав функцию сразу для всех городов, сравним результаты по времени. Для этого напишем функции для тестирования, внесем их в utils.py. Для показательности выведем результат выполнения (время) при условии, что мы запустим сразу по всем городам

In [10]:
print("Тест синхронного анализа:")
test_sync_analysis()

print("\nТест параллельного анализа:")
test_parallel_analysis()

Тест синхронного анализа:
Текущий сезон для города New York: winter

Минимальная температура в сезоне: -14.17410627969581
Максимальная температура в сезоне: 15.461155579740533
Средняя температура в сезоне:  0.189212244512664

Профиль текущего сезона города New York
          city  timestamp  temperature  season  rolling_mean  rolling_std  \
3645  New York 2019-12-25     9.614827  winter      0.108812     5.535941   
3646  New York 2019-12-26    10.071146  winter      0.659973     6.124706   
3647  New York 2019-12-27     0.164934  winter      0.549329     4.978215   
3648  New York 2019-12-28     1.595547  winter      0.392490     4.532006   
3649  New York 2019-12-29     2.356468  winter      0.409757     3.910796   

      is_anomaly                trend  
3645       False  Положительный тренд  
3646       False  Положительный тренд  
3647       False  Положительный тренд  
3648       False  Положительный тренд  
3649       False  Положительный тренд  
Текущий сезон для города London

Результаты стали хуже :(
    
1. Среднее время выполнения синхронного анализа на один город: 0.01 секунд
Общее время выполнения синхронного анализа: 0.15 секунд

2. Среднее время выполнения параллельного анализа на один город: 0.15 секунд
Общее время выполнения параллельного анализа: 2.18 секунд

У нас обработка одного города выполняется быстро (за 0.01 секунды), время на создание и управление процессами перевешивает пользу от их использования

## Монтиоринг текущей температуры

### Анализ текущей температуры

Напишем функцию, которая будет получать текущую температуру для выбранного города, а также определять, является ли текщуая температура нормальной

In [11]:
current_temp('Beijing')

Текущая температура: 270.09 K / -3.06 °C
2.912580021298937
0.5997206059044432
Текущая погода ниже нормы для текущего сезона


Протестируем решение на данных в задании городах

In [12]:
cities = ["Berlin", "Cairo", "Dubai", "Beijing", "Moscow"]

print("Проверяем текущую погоду для 5 городов:")
for city in cities:
    print("_" * 50)
    print(f"Город: {city}")
    current_temp(city)

Проверяем текущую погоду для 5 городов:
__________________________________________________
Город: Berlin
Текущая температура: 278.99 K / 5.84 °C
2.740052074393009
0.9872408764284175
Текущая погода выше нормы для текущего сезона
__________________________________________________
Город: Cairo
Текущая температура: 294.57 K / 21.42 °C
17.492500398384617
0.7902977134136651
Текущая погода выше нормы для текущего сезона
__________________________________________________
Город: Dubai
Текущая температура: 296.11 K / 22.96 °C
22.66028803316494
0.6564470372968394
Погода нормальна для текущего сезона
__________________________________________________
Город: Beijing
Текущая температура: 270.09 K / -3.06 °C
2.912580021298937
0.5997206059044432
Текущая погода ниже нормы для текущего сезона
__________________________________________________
Город: Moscow
Текущая температура: 271.33 K / -1.82 °C
-5.55730240663585
0.913188162327086
Текущая погода выше нормы для текущего сезона


Получилось так, что небольшие аномалии обнаружены в Пекине (как и, видимо, предполагалось заданием) и в Дубае (связываю это с тем, что выполняю эту ячейку под ночь - температура сейчас будет ниже). Нет аномалии в Москве, но будто бы в эти дни в Москве аномально тепло для декабря (предполагаемая аномалия в данных плюс реально аномалия дает отсутствие аномалии так сказать)

### Асинхронный метод, сравнение

Попробуем теперь асинхронный метод и сравним работоспобность, вызвав функцию сразу для всех городов, сравним результаты по времени. Для этого напишем функции для тестирования, внесем их в utils.py. Для показательности выведем результат выполнения (время) при условии, что мы запустим сразу по всем городам

In [13]:
print("Тест синхронной функции:")
test_sync_temp()

print("\nТест асинхронной функции:")
await test_async_temp()

Тест синхронной функции:
Текущая температура: 264.51 K / -8.64 °C
3.9604946379313795
0.8114271458875297
Текущая погода ниже нормы для текущего сезона
Текущая температура: 279.36 K / 6.21 °C
6.681051479970328
0.8355664391943162
Погода нормальна для текущего сезона
Текущая температура: 280.49 K / 7.34 °C
6.688862602713229
0.78202448371885
Погода нормальна для текущего сезона
Текущая температура: 277.08 K / 3.93 °C
9.120234813557724
0.8488253650739982
Текущая погода ниже нормы для текущего сезона
Текущая температура: 271.33 K / -1.82 °C
-5.55730240663585
0.913188162327086
Текущая погода выше нормы для текущего сезона
Текущая температура: 296.58 K / 23.43 °C
13.871225633971573
0.5098734886868355
Текущая погода выше нормы для текущего сезона
Текущая температура: 278.99 K / 5.84 °C
2.740052074393009
0.9872408764284175
Текущая погода выше нормы для текущего сезона
Текущая температура: 270.09 K / -3.06 °C
2.912580021298937
0.5997206059044432
Текущая погода ниже нормы для текущего сезона
Текуща

Стало сильно лучше! 

1. Среднее время выполнения синхронно на один город: 0.23 секунд

    Общее время выполнения синхронно: 3.41 секунд

2. Среднее время выполнения асинхронно на один город: 0.03 секунд

    Общее время выполнения асинхронно: 0.48 секунд

In [16]:
!python3 dashboard.py

2024-12-22 16:04:39.486 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-22 16:04:39.566 
  command:

    streamlit run dashboard.py [ARGUMENTS]
2024-12-22 16:04:39.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-22 16:04:39.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-22 16:04:39.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-22 16:04:39.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-22 16:04:39.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-22 16:04:39.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored 